In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

In [44]:
ob_df = pd.read_csv('/Users/seanyu_mac/Downloads/아워박스_통합관리_조회_초안 - 시트1 (5).csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2024-04-15~2025-04-15.xlsx')

In [46]:
ob_df['주문번호'] = ob_df['주문번호'].apply(lambda x: x[:5])

In [48]:
fs_df['주문 번호'] = fs_df['주문 번호'].apply(lambda x: x[:5])

In [50]:
ob_df['주문일시'] = pd.to_datetime(ob_df['주문일시'])
ob_df['주문일시'] = ob_df['주문일시'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')) 

In [52]:
ob_df['도서산간여부'] = ob_df['도서산간여부'].str.replace("<span class='square square-gray'>","")
ob_df['도서산간여부'] = ob_df['도서산간여부'].str.replace("</span>","")
ob_df['도서산간여부'].unique().tolist()

['일반']

In [54]:
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-primary'>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("</span>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-warning'>","")
ob_df['주문서상태'].unique().tolist()

['정상', '반품']

In [56]:
ob_df['품목명'] = ob_df['품목명'].apply(lambda x: x.split('</span> ')[1].split('</')[0].strip())
ob_df['품목명'].unique().tolist()

['[YDY] 퓨어 리포좀 비타민C (8EA)',
 '[YDY] 퓨어 레드큐민 (3EA)',
 '[YDY] 파이토 레드큐민 (3EA)',
 '[파이토알엑스] 이뮨 빌베리 (8EA)',
 '[파이토알엑스] 이뮨 빌베리 (4EA)',
 '[YDY] 퓨어 리포좀 비타민C (2EA)',
 '[파이토알엑스] 이뮨 빌베리 (2EA)',
 '[YDY] 오큐클리어 (6EA)',
 '[YDY] 액티브비큐텐',
 '[YDY] 퓨어 리포좀 비타민C (4EA)',
 '[YDY] 퓨어 레드큐민 (12EA)',
 '[YDY] 썬 비타민D3 & K2_R1',
 '[YDY] 퓨어 리포좀 비타민C (12EA)',
 '[YDY] 퓨어 리포좀 글루타치온',
 '[파이토알엑스] 라이버 맥주효모 (4EA)',
 '[YDY] 오큐클리어 (12EA)',
 '[YDY] 파이토 레드큐민 (12EA)',
 '[파이토알엑스] 라이버 맥주효모 (8EA)',
 '[YDY] 퓨어 레드큐민 (6EA)',
 '[YDY] 오큐클리어 (3EA)',
 '[YDY] 포스트바이오틱스 GX',
 '[YDY] 프로바이오 에스엘비(SLB)',
 '[YDY] 마그듀오 K2_R1',
 '[파이토알엑스] 라이버 맥주효모 (12EA)',
 '[YDY] 폴라초임계오메가3',
 '[YDY] 오큐클리어 (2EA)',
 '[YDY] 퓨어 리포좀 비타민C',
 '[YDY] 풀스펙 멀티비타민',
 '[YDY] 파이토 레드큐민 (6EA)']

In [58]:
fs_df['결제 완료일'] = fs_df['결제 완료일'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [60]:
temp_df = pd.merge(ob_df, fs_df[['결제 완료일', '옵션명', '상품명', '상품 번호']], left_on=['주문일시','채널상품명'], right_on=['결제 완료일','옵션명'], how='left')

In [62]:
temp_df.columns.tolist()
temp_df['채널상품명'] = temp_df['상품명']
temp_df['옵션1'] = temp_df['옵션명']
temp_df['채널아이디'] = temp_df['상품 번호']

In [64]:
temp_df[temp_df['채널상품명'].isna()]
temp_df.loc[249, '채널상품명'] = '[공동구매] 베이지약사X 파이토알엑스 이뮨 빌베리 / 2박스 4박스 8박스'

In [66]:
del temp_df['결제 완료일']
del temp_df['옵션명']
del temp_df['상품명']
del temp_df['상품 번호']

In [68]:
temp_df['구분'] = ''
con1 = temp_df['채널상품명'].str.contains('임직')
con2 = temp_df['채널상품명'].str.contains('공동구매')
con3 = temp_df['채널상품명'].str.contains('스페셜')
temp_df.loc[con1, '구분'] = '임직원몰 상품'
temp_df.loc[con2, '구분'] = '공동구매'
temp_df.loc[con3, '구분'] = '친한구매'

In [70]:
before = ob_df.index.size
after = len(temp_df['채널상품명'])
before == after

True

In [72]:
temp_df.to_excel(f'/Users/seanyu_mac/Downloads/아워박스_상품명_옵션명_업데이트_{datetime.now()}.xlsx')